In [1]:
# Functions: Analyse the evoltion of average positive and negative
import numpy as np
import pandas as pd
import dask.dataframe as dd
import igraph as ig
import matplotlib.pyplot as plt
import time
from fun.fun import *

pd.set_option('display.width', 500)

c:\Users\stirl\miniconda3\envs\network_env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
# Filenames
dataset_fn = "../dataset/TweetsCOV19.tsv"
plot_fn = "../images/plots/transitivity_evolution.png"

In [99]:
# -> IN : Read Edge List
edges_fn = "../data/edges.parquet"
print("reading edges ... ", end='')
start = time.time()
df = dd.read_parquet("../data/edges.parquet")
df[['source', 'target']] = df[['source', 'target']].astype('int32')
end = time.time()
print("read {:_} lines (took {:.1f}s)".format(len(df), (end-start)))
print(df.dtypes)
df.head()

reading edges ... read 684_732_453 lines (took 0.3s)
source    int64
target    int64
dtype: object


,source,target
0,13,103151
1,13,214293
2,103151,214293
3,13,138731
4,13,42023


In [4]:
# -> IN : Load tweets dataframe
tw = get_filtered_tweets_dataframe(dataset_fn)
print("Loaded {:_} tweets".format(len(tw)))
tw.head()

Importing dataset from tsv file ...read 8_077_794 lines (took 59.9s)
Converting timestamp column
Filtering desired columns and between desired dates ... 8_077_794 rows in dataframe
Parsing hashtags and positive/negative sentiments
filtering for tweets that contain hashtags ... 8_077_794 rows in dataframe
Loaded 462_901 tweets


,Username,Timestamp,Hashtags,Sentiment_pos,Sentiment_neg
Tweet Id,,,,,
1200928806757752833,83c182d0ee195dc692900d7ff7328171,2019-12-01 00:05:14+00:00,"[rgvwx, rgv, txwx, spi]",1,-2
1200931476642115586,73c3e83574f1eb0a5201c71cf46c732c,2019-12-01 00:15:50+00:00,[KeDezembaBoss],2,-4
1200941770835521536,4efca14ff68aa1edd9aea0c78c615ef2,2019-12-01 00:56:45+00:00,"[fame., FreeBritney]",2,-2
1200948819015454721,a53fd620b4178d04a1d2c4e37e85e0ae,2019-12-01 01:24:45+00:00,[FakeNews],1,-2
1200955132944560135,215a5a37e161f7ceaa0c532e73f62c40,2019-12-01 01:49:51+00:00,[KabukiPolitics],1,-1


In [4]:
# -> IN : Read list of nodes
nodes = pd.read_csv('../data/node_ids.csv', index_col='twitter id')
print("Number of nodes: {:_}".format(len(nodes)))
nodes.head()

Number of nodes: 410_885


,index
twitter id,
1200927495186505729,0
1200927503201816576,1
1200927507828097026,2
1200927511087067136,3
1200927514216062976,4


In [96]:
# Compute timestamp bins
steps = 100
ts_min, ts_max = min(tw['Timestamp']), max(tw['Timestamp'])
ts_inc = (ts_max - ts_min) / 100
ts_bins = [ ts_min + i*ts_inc for i in range(steps+1) ]

print("Min timestamp:", ts_min)
print("Max timestamp:", ts_max)
print("Timestamp increment:", ts_inc)

Min timestamp: 2019-12-01 00:00:01+00:00
Max timestamp: 2020-02-29 23:59:49+00:00
Timestamp increment: 0 days 21:50:23.880000


In [70]:
#df
#for i in range(20):
i = 42023
index = nodes.iloc[i]['index']
src_edges = df[df['source'] == index]
#src_edges = src_edges.rename(columns={'target' : 'node'})
print(src_edges.head(10))
tgt_edges = df[df['target'] == index]
#tgt_edges = tgt_edges.rename(columns={'source' : 'node'})
#tgt_edges.drop('target', axis=1, inplace=True)
print(tgt_edges.head())
#print(tgt_edges.T)
#print(df.loc[0])

    source  target
32   42023   70117
33   42023   43183
34   42023   41253
35   42023   44918
36   42023   73474
37   42023   96476
38   42023  104557
39   42023   69055
40   42023  156588
41   42023  149960
   source  target
4      13   42023


In [91]:
#src_edges['target'].append(tgt_edges['source'])
df = pd.DataFrame()
row = pd.concat([src_edges['target'], tgt_edges['source']])
#df = pd.concat([df, row.transpose()], ignore_index=True)
#df
row.transpose()

32     70117
33     43183
34     41253
35     44918
36     73474
37     96476
38    104557
39     69055
40    156588
41    149960
42    156404
43    172186
44    138623
45    153737
46    204783
47    155933
48    178341
49    186975
50    127556
51    138824
52    303271
53    383384
54    410354
55    358554
56    307711
57    282364
58    391316
4         13
dtype: int64

In [98]:
ts = pd.read_csv('test.csv', names=range(20), index_col=0)
ts
for _, row in ts.iterrows():
    print(len(row))

19
19
19
19
19
19
